In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer


def pegar_dataframe_por_num_resposta(num_resposta: int):
    return pd.read_csv('Perguntas 2018+ - {} respostas.csv'.format(num_respostas))#%%



In [5]:
for num_respostas in range(5, 6):
    print("\n\n - - - - - Começando teste com {} respostas - - - - - \n\n".format(num_respostas))
    df = pegar_dataframe_por_num_resposta(num_respostas)
    
    # Filtrando apenas com perguntas com resposta escolhida
    df = df[df['alvo']!=-1]
    
    # Separando as colunas de entrada da rede
    colunas_respostas = ['pontos_resposta', 'tempo_resposta', 'user_reputation', 'maestria_tags']
    colunas_relevantes = [c + str(resposta) for resposta in range(0, num_respostas) for c in colunas_respostas]
    x = df[colunas_relevantes]

    # Separando as colunas com as respostas da rede
    y = df['alvo']
    
    # Preparamos o imputer, definindo variáveis em branco para -1
    imputer = SimpleImputer(missing_values=np.NaN, strategy='constant', fill_value=-1)
    imputer.fit(x)
    x = imputer.transform(x)

    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15)

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)

    scaler = StandardScaler()
    scaler.fit(X_test)
    X_test = scaler.transform(X_test)

    
    print('Numero linhas Treino: ' + str(len(X_train)))
    print('Numero linhas Test: ' + str(len(X_test)))
    
    # Cria modelo
    # Mudamos para learning rate adaptativo, tornando os 'saltos' de cada epoch diminuirem conforme o modelo deixar de melhorar
    mlp = MLPClassifier(hidden_layer_sizes=(num_respostas*10, num_respostas*10, num_respostas*10, num_respostas*10), learning_rate='adaptive', max_iter=2500)
    mlp.fit(X_train, y_train)

    y_pred_train = mlp.predict(X_train)

    cm_treinamento = confusion_matrix(y_train, y_pred_train)
    
    print('\nConjunto de treinamento:\n')
    print('Matriz Confusão:\n' + str(cm_treinamento) + "\n")
    print(classification_report(y_train, y_pred_train))

    y_pred = mlp.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    
    print('\nConjunto de testes:\n')
    print('Matriz Confusão:\n' + str(cm) + "\n")
    print(classification_report(y_test, y_pred))

    print("\n\n - - - - - Terminado teste com {} respostas - - - - - \n\n".format(num_respostas))




 - - - - - Começando teste com 5 respostas - - - - - 


Numero linhas Treino: 644521
Numero linhas Test: 113739

Conjunto de treinamento:

Matriz Confusão:
[[89121 10094  9219 11053  9420]
 [ 8398 90889 10016 11223  7938]
 [ 9701  8742 90457 11062  9259]
 [ 8056  8273  8063 98383  6240]
 [ 9275 11153  9444 13609 85433]]

              precision    recall  f1-score   support

         0.0       0.72      0.69      0.70    128907
         1.0       0.70      0.71      0.71    128464
         2.0       0.71      0.70      0.71    129221
         3.0       0.68      0.76      0.72    129015
         4.0       0.72      0.66      0.69    128914

    accuracy                           0.70    644521
   macro avg       0.71      0.70      0.70    644521
weighted avg       0.71      0.70      0.70    644521


Conjunto de testes:

Matriz Confusão:
[[16475  1160  1759  1458  2005]
 [ 2399 12542  2751  2379  2523]
 [ 1733  1027 16829  1424  1860]
 [ 2097  1039  2330 15475  1700]
 [ 1571  1102  